In [413]:
from pathlib import Path
import os
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import time

In [414]:
dataPath = "F:/project/navigationParadigm/MorrisMaze/data/"
dataFolder = ["ifADo Data","ifADo_2ndRound"]
saveDataPath = 'F:/project/navigationParadigm/MorrisMaze/analyzeBehData/'
dataPath_exl = "F:/project/navigationParadigm/MorrisMaze/data/"
exlName = ["Demographics_04092020_IFaDo.xlsx","Demographic_data_v2.xlsx"]

In [415]:
print(dataPath)

F:/project/navigationParadigm/MorrisMaze/data/


In [425]:
subCont = -1
behInfo = []
for folderNo in range(1):#len(dataFolder)):
    mypath = dataPath+dataFolder[folderNo]+'/'
    print(mypath)
    files = os.listdir(Path(mypath))
    subjNoAll = len(files)
    tmpExlName = dataPath_exl+exlName[folderNo]
    print(tmpExlName)
    File_1 = pd.read_excel(tmpExlName)
    #print(File_1)
    #df = pd.DataFrame(File_1, columns=[File_1.columns[0]])
    #df_age = pd.DataFrame(File_1, columns=[File_1.columns[1]])
    #df_genotype = pd.DataFrame(File_1, columns=[File_1.columns[2]])
    #print(df)
    File_1.info()
    #for ind in File_1.index:
    #    print(File_1[File_1.columns[0]][ind], File_1[File_1.columns[2]][ind])
    for subjNo in range(subjNoAll):
        subCont += 1
        subName = files[subjNo]
        print(subName)
        start_time = time.time()
        
        if "_m_" in subName:
            sex = 'm'
        elif "_f_" in subName:
            sex = 'f'
        
        if folderNo == 0:
            matched_indexes = []
            length = len(File_1)
            for ind in File_1.index:
                if subName[6:] == File_1[File_1.columns[0]][ind]:
                    matched_indexes.append(ind)

            print(matched_indexes)
            age = File_1[File_1.columns[1]][matched_indexes[0]]
            genotype = File_1[File_1.columns[3]][matched_indexes[0]]

            #print(behInfo)
            
            #behInfo.append({
            #    'sex': sex,
            #    'age': age,
            #    'genotype': genotype,
            #    'subjID': subName
            #})
            
            tmpFileName = mypath + subName + '/session_0/log.txt'
            
            fid = open(tmpFileName, 'r')
            fid_text = pd.read_csv(fid, delimiter='\t', header=None, low_memory=False)
            
            # Trigger information
            eventLineNo = fid_text[fid_text[2] == 'bigEvent'].index
            eventName = fid_text.loc[eventLineNo, 3]
            eventTime = fid_text.loc[eventLineNo, 0].astype(float)
            
            startRealExperimentLineNo = fid_text[(fid_text[2] == 'bigEvent') & (fid_text[3] == 'startRealExperimentTrigger')].index
            trialStartLineNo = fid_text[(fid_text[2] == 'bigEvent') & (fid_text[3] == 'trialStartTrigger')].index
            trialStartLineNo = trialStartLineNo[:-1]
            trialEndLineNo = fid_text[(fid_text[2] == 'bigEvent') & (fid_text[3].str.contains('trialInfo:'))].index
            
            #fid_text[4].str.contains
            
            if ~startRealExperimentLineNo.empty:
                trialStartLineNo = [val for val in trialStartLineNo if val >= startRealExperimentLineNo]
                trialEndLineNo = [val for val in trialEndLineNo if val >= startRealExperimentLineNo]
                
            # avatar position informaton
            posXYLineNo = fid_text[(fid_text[3] == 'PandaEPL_avatar') & (fid_text[2] == 'VROBJECT_POS')].index
            posXYTime = fid_text.loc[posXYLineNo, 0].astype(float)
            tmpidx1, tmpidx2 = np.unique(posXYTime, return_index=True)
            posXYTime = tmpidx1
            posXYLineNo = posXYLineNo[tmpidx2]
            
            posXY_heading_LineNo = fid_text[(fid_text[3] == 'PandaEPL_avatar') & (fid_text[2] == 'VROBJECT_HEADING')].index
            posXYTime_heading = fid_text.loc[posXY_heading_LineNo, 0].astype(float)
            posXY_turningSpeedLineNo = fid_text[(fid_text[3] == 'PandaEPL_avatar') & (fid_text[2] == 'MOVINGOBJECT_TURNINGSPEED')].index
            posXYTime_turningSpeed = fid_text.loc[posXY_turningSpeedLineNo, 0].astype(float)
            posXY_linearSpeedLineNo = fid_text[(fid_text[3] == 'PandaEPL_avatar') & (fid_text[2] == 'MOVINGOBJECT_LINEARSPEED')].index
            posXYTime_linearSpeed = fid_text.loc[posXY_linearSpeedLineNo, 0].astype(float)
            
            posXY = np.zeros((len(posXYTime), 2))
            posXY_heading = np.full((len(posXYTime), 1), np.nan)
            posXY_tuningSpeed = np.full((len(posXYTime), 1), np.nan)
            posXY_linearSpeed = np.full((len(posXYTime), 1), np.nan)
            
            for idx in range(len(posXYTime)):
                tmptext = fid_text.loc[posXYLineNo[idx], 4]
                tmpText1 = tmptext.replace('(', ',').replace(')', ',').split(',')
                posXY[idx, :] = [float(tmpText1[1]), float(tmpText1[2])]
                
                tmpidx = np.where(posXYTime_heading == posXYTime[idx])[0]
                if tmpidx.size > 0:
                    tmptext = fid_text.loc[posXY_heading_LineNo[tmpidx[0]], 4]
                    posXY_heading[idx] = float(tmptext)
                    
                tmpidx = np.where(posXYTime_turningSpeed == posXYTime[idx])[0]
                if tmpidx.size > 0:
                    tmptext = fid_text.loc[posXY_turningSpeedLineNo[tmpidx[0]], 4]
                    posXY_tuningSpeed[idx] = float(tmptext)
                    
                tmpidx = np.where(posXYTime_linearSpeed == posXYTime[idx])[0]
                if tmpidx.size > 0:
                    tmptext = fid_text.loc[posXY_linearSpeedLineNo[tmpidx[0]], 4]
                    posXY_linearSpeed[idx] = float(tmptext)
                    
                    
            #print('step1')
            posXY_heading2 = np.full((len(posXYTime),1), np.nan)
            diffY = posXY[1:,1] - posXY[0:-1,1]
            diffX = posXY[1:,0] - posXY[0:-1,0]
            posXY_heading2[1:,0] = np.arctan2(diffY, diffX)
            for i in range(1, len(posXY_heading2)):
                if posXY_heading2[i] == 0:
                    posXY_heading2[i] = posXY_heading2[i - 1]
            
            

            trialNoAll = len(trialStartLineNo)
            trialInfo = []
            for trialNo in range(trialNoAll):
                #print(trialNo)
                tmpidx = np.where((eventLineNo > trialStartLineNo[trialNo]) & (eventLineNo < trialEndLineNo[trialNo]))[0]
                #print(tmpidx)
                test = np.array(eventName)
                eventNames = test[tmpidx]
                #print(eventNames)
                
                trial = {}
                startToTarget = trialStartLineNo[trialNo]
                startSearchTime = float(fid_text.loc[startToTarget, 0])
                trial['startTime'] = startSearchTime

                collectTarget = eventLineNo[tmpidx[eventNames == 'collectTargetTrigger']][0]
                collectTargetTime = float(fid_text.loc[collectTarget, 0])
                trial['collectTargetTime'] = collectTargetTime
                
                tmpXY_idx = np.where((posXYLineNo>startToTarget) & (posXYLineNo<collectTarget))[0]
                trial['posXYTime_start_collectTarget'] = posXYTime[tmpXY_idx]
                trial['posXY_start_collectTarget'] = posXY[tmpXY_idx,:]
                trial['posXY_heading_start_collectTarget'] = posXY_heading2[tmpXY_idx]
                trial['posXY_tuningSpeed_start_collectTarget'] = posXY_tuningSpeed[tmpXY_idx]
                trial['posXY_linearSpeed_start_collectTarget'] = posXY_linearSpeed[tmpXY_idx]
                
                instrToFindTarget = eventLineNo[tmpidx[eventNames == 'instrToFindTargetTrigger']][0]
                instrToFindTargetTime = float(fid_text.loc[instrToFindTarget, 0])
                trial['instrToFindTargetTime'] = instrToFindTargetTime
                
                if 'preinstrToFindTargetTrigger' in eventNames:
                    trial['trialType'] = 'lure'
                    tmpXY_idx = np.where((posXYLineNo>collectTarget) & (posXYLineNo<instrToFindTarget))[0]
                    trial['posXYTime_collectTarget_collectLure'] = posXYTime[tmpXY_idx]
                    trial['posXY_collectTarget_collectLure'] = posXY[tmpXY_idx,:]
                    trial['posXY_heading_collectTarget_collectLure'] = posXY_heading2[tmpXY_idx]
                    trial['posXY_tuningSpeed_collectTarget_collectLure'] = posXY_tuningSpeed[tmpXY_idx]
                    trial['posXY_linearSpeed_collectTarget_collectLure'] = posXY_linearSpeed[tmpXY_idx]
                else:
                    trial['trialType'] = 'teleport'
                
                
                startToDeliverTarget = eventLineNo[tmpidx[np.where(eventNames=='findTargetTrigger')[0]]][0]
                startToDeliverTargetTime = float(fid_text.loc[startToDeliverTarget,0])
                trial['startToDeliverTargetTime'] = startToDeliverTargetTime
                
                dropTarget = eventLineNo[tmpidx[np.where(eventNames=='dropTargetTrigger')[0]]][0]
                dropTargetTime = float(fid_text.loc[dropTarget,0])
                trial['dropTargetTime'] = dropTargetTime
                
                
                tmpXY_idx = np.where((posXYLineNo>startToDeliverTarget) & (posXYLineNo<dropTarget))[0]
                trial['posXYTime_startToDeliver_dropTarget'] = posXYTime[tmpXY_idx]
                trial['posXY_startToDeliver_dropTarget'] = posXY[tmpXY_idx,:]
                trial['posXY_heading_startToDeliver_dropTarget'] = posXY_heading2[tmpXY_idx]
                trial['posXY_tuningSpeed_startToDeliver_dropTarget'] = posXY_tuningSpeed[tmpXY_idx]
                trial['posXY_linearSpeed_startToDeliver_dropTarget'] = posXY_linearSpeed[tmpXY_idx]
                
                displayPic = eventLineNo[tmpidx[eventNames == 'displayPicTrigger']][0]
                displayPicTime = float(fid_text.loc[displayPic,0])
                choosePic = eventLineNo[tmpidx[eventNames == 'choosePicTrigger']][0]
                choosePicTime = float(fid_text.loc[choosePic,0])
                trial['RT_itemMemory'] = choosePicTime - displayPicTime

                
                tmpText = fid_text.loc[trialEndLineNo[trialNo],3]
                tmpText1 = tmpText.split(",")
                tmpText2 = tmpText1[1].split(":");
                trial['targetPic'] = tmpText2[1]
                
                tmpText2 = tmpText1[2].split("[")
                tmpx = float(tmpText2[1])
                tmpText2 = tmpText1[3].split("]")
                tmpy = float(tmpText2[0])
                trial['targetLoc'] = [tmpx,tmpy]
                
                
                tmpText2 = tmpText1[4].split(":")
                targetPicPos = tmpText2[1]
                tmpText2 = tmpText1[5].split(":")
                responsePicPos = tmpText2[1]
                if targetPicPos==responsePicPos:
                    trial['picMemory'] = 1
                else:
                    trial['picMemory'] = 0
                
                tmpText2 = tmpText1[6].split("[")
                tmpx = float(tmpText2[1])
                tmpText2 = tmpText1[7].split("]")
                tmpy = float(tmpText2[0])
                trial['startSearchLoc'] =  [tmpx,tmpy]
                
                tmpText2 = tmpText1[10].split("(")
                tmpx = float(tmpText2[1]);
                tmpy = float(tmpText1[11]);
                trial['dropLoc'] =  [tmpx,tmpy]
                
                trial['targetLoc_adjust'] = trial['posXY_start_collectTarget'][-1,:]
                trial['startSearchLoc_adjust'] = trial['posXY_startToDeliver_dropTarget'][0,:]

                
                trialInfo.append(trial)
                
            behInfo.append({
                'sex': sex,
                'age': age,
                'genotype': genotype,
                'subjID': subName,
                'trialInfo':trialInfo
            })
            print(time.time() - start_time, "seconds")
np.save(saveDataPath + 'behInfo_ifado.npy', behInfo)

F:/project/navigationParadigm/MorrisMaze/data/ifADo Data/
F:/project/navigationParadigm/MorrisMaze/data/Demographics_04092020_IFaDo.xlsx
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Code      71 non-null     object 
 1   Age       71 non-null     int64  
 2   Sex       71 non-null     object 
 3   APOE      70 non-null     object 
 4   rs429358  70 non-null     float64
 5   rs7412    70 non-null     float64
dtypes: float64(2), int64(1), object(3)
memory usage: 3.5+ KB
IFADo_f_01Jul2020_sub1
[31]
70.35722470283508 seconds
IFADo_f_01Jul2020_sub2
[32]
65.79426622390747 seconds
IFADo_f_01Jul2020_sub3
[33]
140.7653877735138 seconds
IFADo_f_02Mar2020_sub1
[23]
109.34694147109985 seconds
IFADo_f_02Mar2020_sub2
[24]
90.83326721191406 seconds
IFADo_f_03Aug2020_sub1
[48]
114.02631282806396 seconds
IFADo_f_03Mar2020_sub1
[25]
143.86365365982056 seconds
IFADo_

In [426]:
del behInfo

In [427]:
saveDataPath = 'F:/project/navigationParadigm/MorrisMaze/analyzeBehData/'
trialInfo = np.load(saveDataPath + 'behInfo_ifado.npy', allow_pickle=True)
print(trialInfo[0]['trialInfo'][0]['startTime'])

1593587567.932
